In [1]:
import pandas as pd
import numpy as np
import pickle
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
import warnings
from tqdm.notebook import tqdm
import random
from scipy.stats import norm
warnings.simplefilter(action='ignore', category=FutureWarning or RuntimeWarning)
DATA_PATH = "../../../data/"

### CB 모델의 성능을 검증한다.

In [2]:
# 메모리상에 여유를 위해 pickling 하는 함수
def pickling(arg_object, arg_file_name):
    with open(f'{arg_file_name}.pkl','wb') as pickle_file:
        pickle.dump(arg_object, pickle_file)       
    print(f"{arg_file_name}.pkl로 pickling 완료")

In [3]:
# 저장한 pkl 파일을 불러오는 함수
def test_pkl(name):
    test = None
    with open(f'{name}.pkl','rb') as pickle_file:
        test = pickle.load(pickle_file)
    return test

In [4]:
# pearson table 을 쉽게 만들기 위해 데이터를 가공후 data_for_pearson.parquet.gzip 으로 저장
df = pd.read_csv(DATA_PATH + "2019-Oct.csv")
df.dropna(inplace = True)
df.drop(columns = ["event_time", "category_id", "user_session"], inplace = True)
df = df[df["event_type"] == "view"]
df.reset_index(drop = True, inplace = True)
df["category_code_0"] = df["category_code"].apply(lambda x : x.split(".")[0])
df.to_parquet(DATA_PATH + "data_for_pearson.parquet.gzip")

In [5]:
def make_pearson_table(target_category):
    '''
    target_category 가 주어지면 해당 카테고리의 상품들로 pearson table을 만들어서 반환하는 함수
    '''

    # 데이터 불러오기 
    df = pd.read_parquet(DATA_PATH + "data_for_pearson.parquet.gzip", columns = ["product_id", "category_code", "brand", "price", "category_code_0"])

    # 해당 카테고리만 가져오기
    df = df[df["category_code_0"] == target_category]
    df = df.reset_index(drop= True)

    # 카테고리와 브랜드를 합친 category_code+brand 변수 생성
    df["category_code+brand"] = df["category_code"] +  df["brand"].apply(lambda x : "." + x)

    # 제품별로 category_code+brand와 가격의 평균으로 보기
    df = df.groupby("product_id").agg({"category_code+brand" : "unique", "price" : "mean"})
    df = df.reset_index()
    df["category_code+brand"] = df["category_code+brand"].apply(lambda x : x[0])
    
    # 가격평균을 MinMaxScaler 를 이용하여 스케일링하기
    # df_minmax 는 스케일링된 가격평균을 가지고 있는 DataFrame
    scaler = MinMaxScaler()
    df_minmax = scaler.fit_transform(df[["price"]])
    df_minmax = pd.DataFrame(df_minmax, columns=['mMprice'])
    df_minmax.index = df['product_id'].values
    del scaler

    # CountVectorizer 적용
    # sparse matrix 인 countvect 에서 직접 계산하면 더 효율적일 것으로 예상
    vect = CountVectorizer()
    docs = df['category_code+brand'].values
    countvect = vect.fit_transform(docs)
    countvect_df =pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
    countvect_df.index = df['product_id'].values
    del vect, docs, countvect

    # 제품을 index로 가지는 데이터(제품별 특징을 담고있다)
    df = pd.concat([df_minmax, countvect_df], axis= 1)
    del df_minmax, countvect_df

    # 피어슨 유사도 계산
    df = df.T.corr()
    return df

In [6]:
df = pd.read_parquet(DATA_PATH + "data_for_pearson.parquet.gzip", columns = ["category_code_0"])
category_code_list = list(df["category_code_0"].unique())
del df

# 각 카테고리별로 pearson table을 생성하고 저장한다.
for category_code in category_code_list:
    df = make_pearson_table(target_category = category_code)
    pickling(df, f"{category_code}_pearson_table")
    del df

appliances_pearson_table.pkl로 pickling 완료
computers_pearson_table.pkl로 pickling 완료
electronics_pearson_table.pkl로 pickling 완료
apparel_pearson_table.pkl로 pickling 완료
furniture_pearson_table.pkl로 pickling 완료
construction_pearson_table.pkl로 pickling 완료
kids_pearson_table.pkl로 pickling 완료
auto_pearson_table.pkl로 pickling 완료
sport_pearson_table.pkl로 pickling 완료
accessories_pearson_table.pkl로 pickling 완료
medicine_pearson_table.pkl로 pickling 완료
stationery_pearson_table.pkl로 pickling 완료
country_yard_pearson_table.pkl로 pickling 완료


### 각 카테고리별로 pearson 유사도를 계산해서 해당 유저가 가장 많이 본 상품과 비슷한 상품 10개를 추천한다

In [7]:
# product_id 입력받으면 해당 제품의 1차 카테고리를 반환하는 dict 생성
df = pd.read_parquet(DATA_PATH + "data_for_pearson.parquet.gzip", columns = ["product_id", "category_code_0"])
df = df.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=True)
product_id_to_category_code_0 = {product_id : category_code_0 for product_id, category_code_0 in list(zip(df["product_id"], df["category_code_0"]))}
pickling(product_id_to_category_code_0, "product_id_to_category_code_0")
del product_id_to_category_code_0, df

product_id_to_category_code_0.pkl로 pickling 완료


In [4]:
# 조회한 상품의 종류가 2개이상 10개 이하인 유저만 불러오기
df = pd.read_parquet(DATA_PATH + "data_for_pearson.parquet.gzip", columns = ["product_id", "user_id"])
df = df.groupby("user_id").nunique()
df = df[(df["product_id"] >= 2) & (df["product_id"] <= 10)]
lower_user_list = df.index
df = pd.read_parquet(DATA_PATH + "data_for_pearson.parquet.gzip", columns = ["user_id", "product_id", "event_type"])
df = df[df["user_id"].isin(lower_user_list)]  
df = df.reset_index(drop =True)
df = df.groupby(["user_id", "product_id"]).count().reset_index()

In [5]:
# 데이터를 csr_matrix로 만드는 과정입니다.
user_unique = df['user_id'].unique()
product_unique = df['product_id'].unique()
cb_user_to_index = {user:index for index, user in enumerate(user_unique)}
cb_index_to_user = {index:user for index, user in enumerate(user_unique)}
cb_product_to_index = {product:index for index, product in enumerate(product_unique)}
cb_index_to_product = {index:product for index, product in enumerate(product_unique)}
df['user_id'] = df['user_id'].map(cb_user_to_index.get)
df['product_id'] = df['product_id'].map(cb_product_to_index.get)
num_user = df['user_id'].nunique()
num_product = df['product_id'].nunique()
lower_user_item_matrix = csr_matrix((df.event_type, (df.user_id, df.product_id)), shape= (num_user, num_product))

del df, user_unique, product_unique

lower_user_item_matrix

<1177302x51628 sparse matrix of type '<class 'numpy.int64'>'
	with 5058355 stored elements in Compressed Sparse Row format>

### 성능 평가를 위해 한 유저당 임의의 상품 하나의 view기록을 0으로 만듬

In [6]:
# 각 유저마다 랜덤하게 하나씩 0으로 가리는 작업입니다.
samples = []

for user_idx in tqdm(range(num_user)) :
    samples.append((user_idx, random.sample(lower_user_item_matrix[user_idx].nonzero()[1].tolist(), 1)[0]))
    
training_set = lower_user_item_matrix.copy()
test_set = lower_user_item_matrix.copy()

user_inds = [index[0] for index in samples]
item_inds = [index[1] for index in samples]

training_set[user_inds, item_inds] = 0
training_set.eliminate_zeros()

del lower_user_item_matrix

  0%|          | 0/1177302 [00:00<?, ?it/s]

In [7]:
# 유저별로 가장 view 수가 큰 product_id를 가지는 list
input_data = list(np.array(np.argmax(training_set, axis=1)).reshape(-1))
input_data = list(map(cb_index_to_product.get, input_data))

# 가려진 product_id를 가지는 list
label = list(map(cb_index_to_product.get, item_inds))

In [8]:
# 표본으로 Hit rate 검증 하기
df = pd.read_parquet(DATA_PATH + "data_for_pearson.parquet.gzip", columns = ["product_id", "event_type"])
df = df.groupby("product_id").count()
popular_product_id_list = list(df.sort_values("event_type", ascending=False).index[:10])
del df
product_id_to_category_code_0 = test_pkl("product_id_to_category_code_0")
answer_store_by_model = []
answer_store_by_pop = []
sample_size = 10000
for user_index in tqdm(np.random.randint(training_set.shape[1], size=sample_size)):
    input_product_id = input_data[user_index]
    input_category_code = product_id_to_category_code_0[input_product_id]
    pearson_table = test_pkl(f"{input_category_code}_pearson_table")
    viewed_product_index_list = list(np.where(training_set[user_index].toarray()[0] != 0)[0])
    viewed_product_id_list = list(map(cb_index_to_product.get, viewed_product_index_list ))
    pearson_table = pearson_table[~pearson_table.index.isin(viewed_product_id_list)]
    answer_by_model = label[user_index] in list(pearson_table[input_product_id].sort_values(ascending=False).index[:10])
    answer_store_by_model.append(answer_by_model)
    answer_by_pop = label[user_index] in popular_product_id_list
    answer_store_by_pop.append(answer_by_pop)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [9]:
# 귀무가설 : cb 모델을 사용하는것의 정확도와 가장조회수가 높은 것을 추천하는것의 Hit rate보다 작거나 같다.
# 대립가설 : cb 모델을 사용하는것의 정확도가 가장조회수가 높은 것을 추천하는것의 Hit rate보다 크다.

model = sum(answer_store_by_model)/len(answer_store_by_model)
pop = sum(answer_store_by_pop)/len(answer_store_by_pop)
print(f"CB 모델의 Hit rate = {model}")
print(f"Baseline 모델의 Hit rate = {pop}")

pool = (sample_size * (model + pop)) / (sample_size * 2)
Z = (model - pop) / np.sqrt(pool * (1 - pool) * (1/sample_size + 1/sample_size))
print(f"검정통계량 값 = {Z}")
print(f"유의확률 = {1 - norm.cdf(Z)}")

CB 모델의 Hit rate = 0.1314
Baseline 모델의 Hit rate = 0.1157
검정통계량 값 = 3.373647532876431
유의확률 = 0.0003708964188694486


유의수준 0.01 하에서 귀무가설을 기각한다.  
cb 모델을 사용하는것의 Hit rate가 가장조회수가 높은 것을 추천하는것의 Hit rate보다 크다.